In [1]:
from mpvn.model import *
from mpvn.metric import *
import pytorch_lightning as pl
from mpvn.configs import DictConfig
from mpvn.data.grad.lit_data_module import LightningGradDataModule
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

configs = DictConfig()

data_module = LightningGradDataModule(configs)
vocab, word_vocab = data_module.get_vocab()

/home/wicii/miniconda3/envs/grad/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wicii/miniconda3/envs/grad/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: latest is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
checkpoint_callback = ModelCheckpoint(
    save_top_k=3,
    monitor="valid_loss",
    mode="min",
    dirpath="checkpoint",
    filename="mpvn-{epoch:02d}-{valid_loss:.2f}-{valid_per:.2f}",
)
early_stop_callback = EarlyStopping(
    monitor="valid_loss", 
    min_delta=0.00, 
    patience=5, 
    verbose=False, 
    mode="min"
)
logger = TensorBoardLogger("tensorboard", name="Pronunciation for Vietnamese")

trainer = pl.Trainer(accelerator='cpu',
                      logger=logger,
                      max_epochs=configs.max_epochs,
                      callbacks=[checkpoint_callback, early_stop_callback])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [3]:
model = ConformerRNNModel(
    configs=configs,
    num_classes=len(vocab),
    num_words=len(word_vocab),
    vocab=vocab,
    per_metric=WordErrorRate(vocab)
)
trainer.test(model, data_module)

/home/wicii/miniconda3/envs/grad/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Testing DataLoader 0:   0%|          | 0/760 [00:00<?, ?it/s]CC


RuntimeError: mat1 and mat2 shapes cannot be multiplied (7x288 and 144x144)

In [3]:
model = ConformerRNNModel.load_from_checkpoint(
    'Checkpoint/lstm-epoch=05-valid_loss=0.37-valid_per=0.51.ckpt',
    configs=configs,
    num_classes=len(vocab),
    num_words=len(word_vocab),
    vocab=vocab,
    per_metric=WordErrorRate(vocab)
)

SyntaxError: invalid syntax (1169492403.py, line 6)

Testing DataLoader 0:   0%|          | 0/760 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (52x288 and 144x144)

In [5]:
trainer.validate(model, data_module)

Validation DataLoader 0:   0%|          | 0/760 [00:00<?, ?it/s]

UnboundLocalError: local variable 'outputs' referenced before assignment

In [26]:
with open('log.log', 'w') as f:
    for i, attn in enumerate(model.save_attn):
        print(i, attn.softmax(dim=-1), file=f)
        print(i, torch.sum(model.save_context.squeeze(), dim=-1)[i], file=f)